In [ ]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install ipywidgets
#!pip install wandb

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


     |██                              | 225 kB 5.7 MB/s eta 0:00:01

     |████████████████████████████████| 3.5 MB 5.7 MB/s 


     |████████████████████████████████| 67 kB 81.5 MB/s 


     |███████▎                        | 1.5 MB 42.2 MB/s eta 0:00:01

...

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


     |████████████████████████████████| 121 kB 23.0 MB/s 


     |███▏                            | 163 kB 67.9 MB/s eta 0:00:01

     |████████████████████████████████| 1.6 MB 67.9 MB/s 


     |████████████████████████████████| 243 kB 92.2 MB/s 


Import libraries

In [ ]:
import json
from datasets import load_dataset, load_metric, load_from_disk
import pandas as pd
from transformers import T5Model, T5ForConditionalGeneration, T5Tokenizer
from transformers import Adafactor
import torch
from torch import nn
import torch.nn.functional as F
import wandb

Dataclass for preprocessing and creating train and test csv files

In [ ]:
class DataClass:

  def __init__(self, data_dir):
    self.data_dir = data_dir

  def data_csv(self, f, output):

    answers = []
    rewrites = []
    passages = []
    questions = []
    contexts = []

    filepath = self.data_dir+f

    with open(filepath) as fl:
      data = json.load(fl)
      
      for d in data:
        answers.append(d['answer'])
        rewrites.append(d['rewrite'])
        passages.append(d['passage'])
        contexts.append(d['context'])
        questions.append(d['question'])


      data = {'answer':answers, 'passage':passages, 'rewrite':rewrites,
              'context':contexts, 'question':questions}
      df = pd.DataFrame(data)
      df.to_csv(output, index=False)


data = DataClass('/storage/qrecc/')

data.data_csv('qrecc_train.json', 'train.csv')
data.data_csv('qrecc_test.json', 'test.csv')

qrecc = load_dataset('csv', data_files={'train': 'train.csv', 'test': 'test.csv'})

Using custom data configuration default-48e180c5b3892efd


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-48e180c5b3892efd/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Huggingface models and tokenizers

In [ ]:
max_length= 384
batch_size = 2  # 4
dim = 768 # change BERT hidden size to change

pretrained_model = 't5-base'
#pretrained_model = 'google/t5-v1_1-base'

tokenizer = T5Tokenizer.from_pretrained(pretrained_model)
qr_model = T5ForConditionalGeneration.from_pretrained(pretrained_model)
rc_model = T5ForConditionalGeneration.from_pretrained(pretrained_model)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Tokenize dataset

In [ ]:
# tokenize dataset
def tokenize_dataset(batch):
  contexts = tokenizer(batch['context'], batch['question'], padding='max_length',
                       truncation='only_first', max_length=max_length, add_special_tokens=True)
  rewrites = tokenizer(batch['rewrite'], padding='max_length', truncation=True, max_length=max_length, add_special_tokens=True)
  passages = tokenizer(batch['passage'], padding='max_length', truncation=True, max_length=max_length, add_special_tokens=True)
  answers = tokenizer(batch['answer'], padding='max_length', truncation=True, max_length=max_length, add_special_tokens=True)

  batch['ctx_input_ids'] = contexts.input_ids
  batch['rwrt_input_ids'] = rewrites.input_ids
  batch['psg_input_ids'] = passages.input_ids
  batch['ans_input_ids'] = answers.input_ids
 
  batch['ctx_attention_mask'] = contexts.attention_mask
  batch['rwrt_attention_mask'] = rewrites.attention_mask
  batch['psg_attention_mask'] = passages.attention_mask

  return batch


# handle examples with no ansers
def no_ans(x):
  if isinstance(x['answer'], str): return x
  x['answer'] = 'no_ans'
  return x

# removing examples with no context
qrecc = qrecc.filter(lambda x: isinstance(x['context'], str) and isinstance(x['question'], str))

# no answers
qrecc = qrecc.map(no_ans)

# tokenizing
dataset = qrecc.map(
    tokenize_dataset, 
    batch_size = batch_size,
    batched=True,
    remove_columns=['answer', 'passage', 'rewrite', 'context' ,'question']
)


dataset.set_format(
    type='torch', columns=['ctx_input_ids', 'rwrt_input_ids', 'psg_input_ids',
                           'ans_input_ids', 'ctx_attention_mask', 'rwrt_attention_mask',
                           'psg_attention_mask'],)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

  0%|          | 0/9893 [00:00<?, ?ba/s]

  0%|          | 0/2675 [00:00<?, ?ba/s]

Train and test dataloaders

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset['train'], batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset['test'], batch_size=batch_size)

Forward function and function for rolling tensors

In [ ]:
def roll_by_gather(mat, dim, shifts:torch.LongTensor):
    # assumes 2D array
    n_rows, n_cols = mat.shape
    
    if dim == 0:
        #print(mat)
        arange1 = torch.arange(n_rows).view((n_rows, 1)).repeat((1, n_cols)).to(device)
        #print(arange1)
        arange2 = (arange1 - shifts) % n_rows
        #print(arange2)
        return torch.gather(mat, 0, arange2)
    elif dim == 1:
        arange1 = torch.arange(n_cols).view((1,n_cols)).repeat((n_rows,1)).to(device)
        #print(arange1)
        arange2 = (arange1 - shifts) % n_cols
        #print(arange2)
        return torch.gather(mat, 1, arange2)


def forward(batch):

    # context + question input
    ctx_input = batch['ctx_input_ids'].to(device) # QR input
    ctx_attention = batch['ctx_attention_mask'].to(device)

    # gold rewrite input for qr loss
    rwrt_input = batch['rwrt_input_ids']
    # # tokens with indices set to -100 are ignored (masked)
    rwrt_input[rwrt_input == tokenizer.pad_token_id] = -100 
    rwrt_input = rwrt_input.to(device)
    rwrt_attention = batch['rwrt_attention_mask'].to(device)

    # passage input
    psg_input = batch['psg_input_ids'].to(device)
    # need to add sep token at the begining
    # roll by 1 and add column of 1s
    psg_input = torch.roll(psg_input, 1, 1)
    psg_input[:, 0] = 1

    # answer input
    ans_input = batch['ans_input_ids']
    # # tokens with indices set to -100 are ignored (masked)
    ans_input[ans_input == tokenizer.pad_token_id] = -100 
    ans_input = ans_input.to(device)

    # feed context+question input and rewrite label to qr model
    qr_output = qr_model(input_ids=ctx_input, attention_mask=ctx_attention, labels=rwrt_input)

    # logits to be sampled from
    logits = qr_output.logits

    # qr loss
    qr_loss = qr_output.loss

    # gumbel softmax on the logits
    # slice upto actual vocabulary size
    gumbel_output = F.gumbel_softmax(logits, tau=1, hard=True).long()[..., :act_vocab_size]
    print(gumbel_output.requires_grad)

    gumbel_output = gumbel_output.cpu()

    # use to one hot samples (straight through trick) to get vocab ids using dummy vocab
    rc_input = gumbel_output@dummy_vocab
    rc_input = rc_input.to(device)

    del gumbel_output, qr_output, logits, ctx_input, ctx_attention, rwrt_input

    # mask rc input ids with attention mask
    rc_input = torch.mul(rc_input, rwrt_attention)
    # flip the rewrite attention mask, replace 1s with 0s and vice versa
    # now the 1s represent the 'free space' in the rc_input tensor to fit the passages
    flipped_rwrt_mask = torch.fliplr(rwrt_attention)
    flipped_mask = flipped_rwrt_mask.clone()
    flipped_mask[flipped_rwrt_mask == 0] = 1
    flipped_mask[flipped_rwrt_mask == 1] = 0
    # mask passage to extract ids that can fit in the rc_input tensor
    extr_psg = torch.mul(flipped_mask, psg_input)
    # find the shifts for each row of extr_psg
    # this is equal to the number of 1s in each row of rwrt_attention
    # reshape to column vector as required by the custom gather function
    shifts = (rwrt_attention==1).sum(dim=1).reshape(-1, 1) 
    # roll each row by the amount occupied by rc_input in that row
    trunc_psg = roll_by_gather(extr_psg, 1, shifts)
    # add to get rwrt + psg as rc_input
    rc_input = torch.add(rc_input, trunc_psg)
    # create attention mask
    rc_attention = rc_input.clone()
    rc_attention[rc_input != 0] = 1

    del flipped_rwrt_mask, flipped_mask, extr_psg, shifts, trunc_psg, psg_input
    
    rc_loss = rc_model(input_ids=rc_input, attention_mask=rc_attention, labels=ans_input).loss

    #del ans_input, rc_input, rc_attention

    return qr_loss, rc_loss



In [ ]:
def roll_by_gather(mat, dim, shifts:torch.LongTensor):
    # assumes 2D array
    n_rows, n_cols = mat.shape
    
    if dim == 0:
        #print(mat)
        arange1 = torch.arange(n_rows).view((n_rows, 1)).repeat((1, n_cols)).to(device)
        #print(arange1)
        arange2 = (arange1 - shifts) % n_rows
        #print(arange2)
        return torch.gather(mat, 0, arange2)
    elif dim == 1:
        arange1 = torch.arange(n_cols).view((1,n_cols)).repeat((n_rows,1)).to(device)
        #print(arange1)
        arange2 = (arange1 - shifts) % n_cols
        #print(arange2)
        return torch.gather(mat, 1, arange2)


def forward(batch):

    # context + question input
    ctx_input = batch['ctx_input_ids'].to(device) # QR input
    ctx_attention = batch['ctx_attention_mask'].to(device)

    # gold rewrite input for qr loss
    rwrt_input = batch['rwrt_input_ids']
    # # tokens with indices set to -100 are ignored (masked)
    rwrt_input[rwrt_input == tokenizer.pad_token_id] = -100 
    rwrt_input = rwrt_input.to(device)
    rwrt_attention = batch['rwrt_attention_mask'].to(device)

    # passage input
    psg_input = batch['psg_input_ids'].to(device)
    # need to add sep token at the begining
    # roll by 1 and add column of 1s
    psg_input = torch.roll(psg_input, 1, 1)
    psg_input[:, 0] = 1

    # answer input
    ans_input = batch['ans_input_ids']
    # # tokens with indices set to -100 are ignored (masked)
    ans_input[ans_input == tokenizer.pad_token_id] = -100 
    ans_input = ans_input.to(device)

    # feed context+question input and rewrite label to qr model
    qr_output = qr_model(input_ids=ctx_input, attention_mask=ctx_attention, labels=rwrt_input)

    # logits to be sampled from
    logits = qr_output.logits

    # qr loss
    qr_loss = qr_output.loss

    # gumbel softmax on the logits
    # slice upto actual vocabulary size
    gumbel_output = F.gumbel_softmax(logits, tau=1, hard=True)[..., :act_vocab_size]
    #print(gumbel_output)
    print(gumbel_output.shape)

    for i in range(max_length):
        gumbel1
        break

    gumbel_output = gumbel_output.cpu()

    # use to one hot samples (straight through trick) to get vocab ids using dummy vocab
    rc_input = gumbel_output@dummy_vocab
    rc_input = rc_input.to(device)

    del gumbel_output, qr_output, logits, ctx_input, ctx_attention, rwrt_input

    # mask rc input ids with attention mask
    rc_input = torch.mul(rc_input, rwrt_attention)
    # flip the rewrite attention mask, replace 1s with 0s and vice versa
    # now the 1s represent the 'free space' in the rc_input tensor to fit the passages
    flipped_rwrt_mask = torch.fliplr(rwrt_attention)
    flipped_mask = flipped_rwrt_mask.clone()
    flipped_mask[flipped_rwrt_mask == 0] = 1
    flipped_mask[flipped_rwrt_mask == 1] = 0
    # mask passage to extract ids that can fit in the rc_input tensor
    extr_psg = torch.mul(flipped_mask, psg_input)
    # find the shifts for each row of extr_psg
    # this is equal to the number of 1s in each row of rwrt_attention
    # reshape to column vector as required by the custom gather function
    shifts = (rwrt_attention==1).sum(dim=1).reshape(-1, 1) 
    # roll each row by the amount occupied by rc_input in that row
    trunc_psg = roll_by_gather(extr_psg, 1, shifts)
    # add to get rwrt + psg as rc_input
    rc_input = torch.add(rc_input, trunc_psg)
    # create attention mask
    rc_attention = rc_input.clone()
    rc_attention[rc_input != 0] = 1

    del flipped_rwrt_mask, flipped_mask, extr_psg, shifts, trunc_psg, psg_input
    
    rc_loss = rc_model(input_ids=rc_input, attention_mask=rc_attention, labels=ans_input).loss

    #del ans_input, rc_input, rc_attention

    return qr_loss, rc_loss



Validation and Training loop

In [ ]:
num_epochs = 2

device = torch.device('cuda')
qr_model.to(device)
rc_model.to(device)

# load finetuned models
qr_model.load_state_dict(torch.load('/storage/qrecc/models/qr/qr_gen3.pth'))
rc_model.load_state_dict(torch.load('/storage/qrecc/models/rc/rc_gen3.pth'))

qr_model.train()
rc_model.train()

# optimizer
optim = optimizer = Adafactor(
    list(qr_model.parameters())+list(rc_model.parameters()),
    lr=1e-5,
    eps=(1e-30, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.8,
    beta1=None,
    weight_decay=0.0,
    relative_step=False,
    scale_parameter=False,
    warmup_init=False
)

config = {
  "learning_rate": 1e-5,
  "epochs": 2,
  "batch_size": 4,
  "weight_decay": 0.0,
  "temperature": 1
}

#wandb.init(project="e2e-gradients", entity="suicune", reinit=True, config=config)

#wandb.watch(qr_model, log="all", log_freq=100)

# vocabulary size
act_vocab_size = len(tokenizer.get_vocab())
# dummy vocab to get vocab ids after gumbel softmax
dummy_vocab = torch.arange(act_vocab_size).long()


def valid_loss():
  
  qr_epoch_loss = 0
  rc_epoch_loss = 0
  idx = 0

  for batch in test_loader:

    qr_loss, rc_loss = forward(batch)

    qr_epoch_loss += qr_loss.item()
    rc_epoch_loss += rc_loss.item()

    #del ans_input, rc_input, rc_attention
    del qr_loss, rc_loss 

    idx += 1

  print('Valid loss : {}, {}'.format(qr_epoch_loss/idx, rc_epoch_loss/idx))


for epoch in range(1, num_epochs+1):
    
  qr_epoch_loss = 0
  rc_epoch_loss = 0

  idx = 1

  for batch in train_loader:

    qr_loss, rc_loss = forward(batch)

    #total_loss = sum([qr_loss, rc_loss])
    qr_epoch_loss += qr_loss.item()
    rc_epoch_loss += rc_loss.item()

    #total_loss.backward()
    rc_loss.backward()

    if idx % 100 == 0:
      print('epoch {}, batch {}'.format(epoch, idx))

      #grad_dict = {}
      for name, param in rc_model.named_parameters():
        if param.requires_grad:
          print(name, param.grad)

      #wandb.log(grad_dict)

    optim.step()
    optim.zero_grad()
      
    #del ans_input, rc_input, rc_attention
    #del qr_loss, rc_loss, total_loss


    idx += 1


  print('Train loss : {}, {}'.format(qr_epoch_loss/len(train_loader), rc_epoch_loss/len(train_loader)))
  qr_model.eval()
  rc_model.eval()
  valid_loss()
  print('\n')
  qr_model.train()
  rc_model.train()
  torch.save(qr_model.state_dict(), '/storage/qrecc/models/e2e/qr'+str(epoch+3)+'.pth')
  torch.save(rc_model.state_dict(), '/storage/qrecc/models/e2e/rc'+str(epoch+3)+'.pth')

RuntimeError: CUDA out of memory. Tried to allocate 14.00 MiB (GPU 0; 15.75 GiB total capacity; 14.22 GiB already allocated; 7.31 MiB free; 14.86 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

Train loss : 0.3970739206526509, 0.45609857336574516
Valid loss : 0.4757325287272927, 0.5390424355815783

In [ ]:
saved_weights = [0.1, 0.2, 0.3, 0.25]
loaded_weights = torch.tensor(saved_weights)
loaded_weights.requires_grad = True
loaded_weights

tensor([0.1000, 0.2000, 0.3000, 0.2500], requires_grad=True)